In [ ]:
import datetime
from tqdm import tqdm
now = datetime.datetime.now()
print now.year, now.month, now.day, now.hour, now.minute, now.second

In [ ]:
from sklearn.datasets.base import Bunch

In [ ]:
dd = {123: Bunch(seq=[1,1,4,5,6], info=['M', 52]),
      456: Bunch(seq=[2,1,4,5,6], info=['M', 52]),
      678: Bunch(seq=[3,1,4,5,6], info=['M', 52]),
      999: Bunch(seq=[4,1,4,5,6], info=['M', 52]),}

In [ ]:
import multiprocessing as mp
import numpy as np
from multiprocessing import Manager

def f(i, shared_arg=None):
    return 2*np.array(shared_arg[i].seq)

manager = Manager()
shared_dict = manager.dict(dd)
pool = mp.Pool(4)

results = [pool.apply_async(f, [i, shared_dict]) for i in [123, 456, 678, 999]]
results = [p.get() for p in results]
results

In [ ]:
import pandas as pd
import numpy as np
from __future__ import division

In [ ]:
df = pd.read_csv('../tmp/dump_class_0.csv', header=0)
df.head()

In [ ]:
n_jobs = 32

splits = np.array_split(df.values, n_jobs)

In [ ]:
ss[0]

In [ ]:
df.shape

In [ ]:
import cPickle as pkl
import pandas as pd
# import dask.dataframe as dd
# pp = pkl.load(open('../tmp/dump_class_0__raw_data_.pkl', 'rb'))

In [ ]:
import os
mbs_files = filter(lambda x: x.startswith('MBS'), os.listdir('../../../data/'))
print(mbs_files)

In [ ]:
# ___MBS_FILES_DICT__ = dict()
for mbs in mbs_files:
    mbs = os.path.join('..','..','..','data', mbs)
    df = pd.read_csv(mbs, header=0, usecols=['PIN', 'ITEM', 'DOS'])
#     print(df.shape)
#     dd = dd.loc[~dd['ITEM'].isin(pregnancy_items), :]
#     print(dd.shape)
#     ___MBS_FILES_DICT__[mbs] = dd
    print('---')
    break

In [ ]:
df.head()

In [ ]:
df.loc[:, 'DOS'] = pd.to_datetime(df['DOS'], format='%d%b%Y') 

In [ ]:
df.head()

In [ ]:
df[df['PIN'] == 1779689312]['DOS'].idxmin()

In [ ]:
df[['PIN', 'DOS']]

In [ ]:
a = pd.DatetimeIndex(asd.values)
a.year[0]

In [ ]:
np.mean(a.year)

In [ ]:
aa = (0, 1)

In [ ]:
aa[0]

In [ ]:
_dd = pd.read_csv(os.path.join('..', 'mbspbs10pc', 'data', 'drugs_used_in_diabetes.csv'),
                     header=0)
# Fix 6-digit notation
dd = set()  # dd should be a set for performance reasons
for item in _dd.values.ravel():
    if len(item) < 6:
        dd.add(str(0)+item)
    else:
        dd.add(item)

In [ ]:
ddd = ___MBS_FILES_DICT__[k]
ddd = ddd.set_index('PIN')
ddd.head()

In [ ]:
ddd.loc[1779689312, ['DOS', 'ITEM']]

In [ ]:
tmp = pd.read_csv('../../../data/PBS_SAMPLE_10PCT_2009.csv', header=0, nrows=1000, usecols=['PTNT_ID', 'SPPLY_DT'])
tmp.loc[:, 'SPPLY_DT'] = pd.to_datetime(tmp['SPPLY_DT'], format='%d%b%Y')

In [ ]:
tmp['SPPLY_DT'].max()

In [ ]:
tmp['PTNT_ID'].values[-1]

In [ ]:
os.path.split(os.path.join('..', '..', 'asd', 'asd'))

In [ ]:
k = mbs
first_dd = np.where(___MBS_FILES_DICT__[k]['ITEM'].isin(dd))[0][0]
small_mbs_dd[k].iloc[:first_dd,:]

In [ ]:
np.where(___MBS_FILES_DICT__[k]['ITEM'].isin(dd))

In [ ]:
pregnancy_items = set(pd.read_csv(os.path.join('../mbspbs10pc/data', 'pregnancy_items.csv'),
                                  header=0, usecols=['ITEM']).values.ravel())

In [ ]:
pregnancy_items

In [ ]:
np.where(dd['ITEM'].isin(pregnancy_items))

In [ ]:
imap = pd.read_csv('../data/imap.tsv', sep='\t', header=0,
                   usecols=['ITEM', 'BTOS'])

In [ ]:
# dd.join(imap, on='ITEM', how='left')
cc = pd.merge(dd, imap, how='left', on='ITEM')
cc.head()

In [ ]:
dd.shape

In [ ]:
dd.columns

In [ ]:
mbs_dd = ___MBS_FILES_DICT__

In [ ]:
raw_data = dict()

In [ ]:
split = set(splits[0].ravel())

In [ ]:
import time

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
t1 = time.time()
raw_data = dict()

# Pre-filter: keep only the elements of mbs_dd that are in the current split
# this helps in reducing the time of the next step
small_mbs_dd = dict()
for k in tqdm(sorted(mbs_dd.keys())):
    # keep only a subset of the full MBS data
    small_mbs_dd[k] = mbs_dd[k].loc[mbs_dd[k]['PIN'].isin(split)]
    # change format to the right datetime format (this is gonna be useful later)
    small_mbs_dd[k].loc[:,'DOS'] = pd.to_datetime(small_mbs_dd[k]['DOS'], format='%d%b%Y')    
    # and sort by date
    small_mbs_dd[k].sort_values(by='DOS', inplace=True)

# Now track down each patient in the reduced MBS files
for s in tqdm(split):
    tmp = pd.DataFrame(columns=['PIN', 'DOS', 'SPR_RSP'])
    for k in sorted(mbs_dd.keys()):
        tmp = pd.concat((tmp, small_mbs_dd[k].loc[small_mbs_dd[k]['PIN'] == s]))
        
    # evaluate the first order difference and convert each entry in days
    timedeltas = map(lambda x: pd.Timedelta(x).days, tmp['DOS'].values[1:] - tmp['DOS'].values[:-1])
    # then build the sequence as ['exam', idle-days, 'exam', idle-days, ...]
    raw_data[s] = flatten([[str(spr_rsp), dt] for spr_rsp, dt in zip(tmp['SPR_RSP'].values, timedeltas)])
    raw_data[s].append(str(tmp['SPR_RSP'].values[-1]))
    
t2 = time.time()
print(t2-t1)

In [ ]:
import multiprocessing as mp

In [ ]:
mp.pool.ThreadPool()

In [ ]:
!hostname

In [ ]:
os.name

In [ ]:
dd = pkl.load(open('../tmp/dump_class_0__raw_data_.pkl','rb'))

In [ ]:
raw_data = dd['raw_data']

In [ ]:
from collections import Counter
Counter(raw_data[9061269504][::2])

In [ ]:
plt.stem(raw_data[9061269504][1::2])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.arange(1, 11)[1::2]

In [34]:
dd = pd.read_csv('../../../data/MBS_SAMPLE_10PCT_2008.csv', header=0)
# dd['DOS'] = pd.to_datetime(dd['DOS'], format='%d%b%Y')
# dd.sort_values(by='DOS', inplace=True)
# dd

In [35]:
%%time
r1 = dd.loc[dd['PIN'] == 3990912150, :]

CPU times: user 112 ms, sys: 52 ms, total: 164 ms
Wall time: 162 ms


In [36]:
%%time
r2 = dd.query('PIN == 3990912150')

CPU times: user 436 ms, sys: 132 ms, total: 568 ms
Wall time: 270 ms


In [33]:
r2

,PIN,DOS,ITEM
6,3990912150,24APR2008,105
366,3990912150,19APR2008,63331
726,3990912150,04JUN2008,10900
1086,3990912150,20MAY2008,55848
1446,3990912150,09JUN2008,10991
1806,3990912150,05DEC2008,73939
2166,3990912150,09JUN2008,23
2526,3990912150,05DEC2008,66512
2886,3990912150,05DEC2008,66656
3246,3990912150,05DEC2008,66716


In [ ]:
date = dd['DOS'].values[3]
date

In [ ]:
dd[dd['DOS'] > date]

In [ ]:
dd.values[0]

In [ ]:
import pandas as pd

df = pd.DataFrame(data=[[1,2,None],[4,5,None], [4, 0, None]], columns=['a','b','c'])
df

In [ ]:
df['c'].values[0] is None

In [ ]:
df[df.columns.drop('c')]

In [5]:
import numpy as np
import pandas as pd

filename = '/tmp/test.h5'

df = pd.DataFrame(np.arange(10).reshape((5,2)), columns=['A', 'B'])
print(df)
#    A  B
# 0  0  1
# 1  2  3
# 2  4  5
# 3  6  7
# 4  8  9

# Save to HDF5
df.to_hdf(filename, 'data', mode='w', format='fixed')
# del df    # allow df to be garbage collected

# # Append more data
# df2 = pd.DataFrame(np.arange(10).reshape((5,2))*10, columns=['A', 'B'])
# df2.to_hdf(filename, 'data', append=True)

# print(pd.read_hdf(filename, 'data'))

   A  B
0  0  1
1  2  3
2  4  5
3  6  7
4  8  9


In [6]:
pd.read_hdf('/tmp/test.h5')

,A,B
0,0,1
1,2,3
2,4,5
3,6,7
4,8,9


In [11]:
df.columns.append(pd.Index(['C']))

Index([u'A', u'B', u'C'], dtype='object')